In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

class BitcoinDCABot:
    def __init__(self, data_path, investment_amount=100, frequency='monthly'):
        """
        Initialize the DCA bot with historical data and investment parameters
        
        Parameters:
        data_path (str): Path to the historical Binance data CSV
        investment_amount (float): Amount to invest at each interval (in USD)
        frequency (str): 'daily', 'weekly', or 'monthly'
        """
        self.investment_amount = investment_amount
        self.frequency = frequency
        
        # Load and process data
        self.data = self.load_data(data_path)
        
    def load_data(self, data_path):
        """Load and format the Binance dataset"""
        df = pd.read_csv(data_path)
        
        # Convert timestamp to datetime if it's not already
        if not pd.api.types.is_datetime64_any_dtype(df['timestamp']):
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            
        # Set timestamp as index
        df.set_index('timestamp', inplace=True)
        df.sort_index(inplace=True)
        
        return df
    
    def simulate_dca(self):
        """Simulate dollar cost averaging strategy"""
        # Create a copy of the dataframe for results
        df = self.data.copy()
        
        # Resample based on frequency
        if self.frequency == 'daily':
            investment_dates = df.resample('D').first().index
        elif self.frequency == 'weekly':
            investment_dates = df.resample('W').first().index
        elif self.frequency == 'monthly':
            investment_dates = df.resample('M').first().index
        else:
            raise ValueError("Frequency must be 'daily', 'weekly', or 'monthly'")
        
        # Initialize investment tracking
        investments = []
        btc_holdings = 0
        total_invested = 0
        
        # Perform DCA strategy
        for date in investment_dates:
            if date in df.index:
                # Use the closing price for investment
                price = df.loc[date, 'close']
                btc_purchased = self.investment_amount / price
                btc_holdings += btc_purchased
                total_invested += self.investment_amount
                
                investments.append({
                    'date': date,
                    'price': price,
                    'investment': self.investment_amount,
                    'btc_purchased': btc_purchased,
                    'total_btc': btc_holdings,
                    'total_invested': total_invested,
                    'current_value': btc_holdings * price,
                    'roi': (btc_holdings * price / total_invested - 1) * 100 if total_invested > 0 else 0
                })
            
        # Convert to DataFrame
        results = pd.DataFrame(investments)
        
        return results
    
    def compare_with_lump_sum(self, results):
        """Compare DCA strategy with lump sum investment"""
        if results.empty:
            return None
        
        total_invested = results.iloc[-1]['total_invested']
        first_price = results.iloc[0]['price']
        last_price = results.iloc[-1]['price']
        
        lump_sum_btc = total_invested / first_price
        lump_sum_final_value = lump_sum_btc * last_price
        lump_sum_roi = (lump_sum_final_value / total_invested - 1) * 100
        
        dca_final_value = results.iloc[-1]['current_value']
        dca_roi = results.iloc[-1]['roi']
        
        comparison = {
            'Total Invested': total_invested,
            'DCA Final Value': dca_final_value,
            'DCA ROI': dca_roi,
            'Lump Sum Final Value': lump_sum_final_value,
            'Lump Sum ROI': lump_sum_roi,
            'Difference': dca_final_value - lump_sum_final_value,
            'Outperformer': 'DCA' if dca_final_value > lump_sum_final_value else 'Lump Sum'
        }
        
        return comparison
    
    def visualize_results(self, results):
        """Visualize the DCA performance"""
        if results.empty:
            print("No results to visualize")
            return
        
        # Set up the figure
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15), sharex=True)
        
        # Plot Bitcoin price
        ax1.plot(results['date'], results['price'], 'b-')
        ax1.set_title('Bitcoin Price')
        ax1.set_ylabel('USD')
        ax1.grid(True)
        
        # Plot accumulated BTC
        ax2.plot(results['date'], results['total_btc'], 'g-')
        ax2.set_title('Accumulated BTC')
        ax2.set_ylabel('BTC')
        ax2.grid(True)
        
        # Plot portfolio value and total invested
        ax3.plot(results['date'], results['current_value'], 'r-', label='Portfolio Value')
        ax3.plot(results['date'], results['total_invested'], 'k--', label='Total Invested')
        ax3.set_title('Portfolio Value vs. Total Invested')
        ax3.set_ylabel('USD')
        ax3.set_xlabel('Date')
        ax3.legend()
        ax3.grid(True)
        
        plt.tight_layout()
        plt.show()
        
        # ROI over time
        plt.figure(figsize=(12, 6))
        plt.plot(results['date'], results['roi'], 'g-')
        plt.title('Return on Investment Over Time')
        plt.ylabel('ROI (%)')
        plt.xlabel('Date')
        plt.grid(True)
        plt.axhline(y=0, color='r', linestyle='-')
        plt.tight_layout()
        plt.show()


def run_dca_simulation(data_path, investment_amount=100, frequency='monthly'):
    """Run a DCA simulation and display results"""
    bot = BitcoinDCABot(data_path, investment_amount, frequency)
    results = bot.simulate_dca()
    
    # Display summary statistics
    print(f"\n====== DCA SIMULATION SUMMARY ======")
    print(f"Strategy: ${investment_amount} {frequency} investment")
    print(f"Total periods: {len(results)}")
    print(f"Total invested: ${results.iloc[-1]['total_invested']:.2f}")
    print(f"Final BTC holdings: {results.iloc[-1]['total_btc']:.8f}")
    print(f"Final portfolio value: ${results.iloc[-1]['current_value']:.2f}")
    print(f"Overall ROI: {results.iloc[-1]['roi']:.2f}%")
    
    # Compare with lump sum
    comparison = bot.compare_with_lump_sum(results)
    if comparison:
        print("\n====== DCA vs. LUMP SUM COMPARISON ======")
        for key, value in comparison.items():
            if isinstance(value, float):
                print(f"{key}: {value:.2f}")
            else:
                print(f"{key}: {value}")
    
    # Visualize results
    bot.visualize_results(results)
    
    return results, bot


# EXECUTION CODE - Replace with your actual file pathimport pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

class BitcoinDCABot:
    def __init__(self, data_path, investment_amount=100, frequency='monthly'):
        """
        Initialize the DCA bot with historical data and investment parameters
        
        Parameters:
        data_path (str): Path to the historical Binance data CSV
        investment_amount (float): Amount to invest at each interval (in USD)
        frequency (str): 'daily', 'weekly', or 'monthly'
        """
        self.investment_amount = investment_amount
        self.frequency = frequency
        
        # Load and process data
        self.data = self.load_data(data_path)
        
    def load_data(self, data_path):
        """Load and format the Binance dataset"""
        df = pd.read_csv(data_path)
        
        # Convert timestamp to datetime based on the format '2025-02-14 17:52:00'
        if not pd.api.types.is_datetime64_any_dtype(df['timestamp']):
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            
        # Set timestamp as index
        df.set_index('timestamp', inplace=True)
        df.sort_index(inplace=True)
        
        return df
    
    def simulate_dca(self):
        """Simulate dollar cost averaging strategy"""
        # Create a copy of the dataframe for results
        df = self.data.copy()
        
        # Resample based on frequency
        if self.frequency == 'daily':
            investment_dates = df.resample('D').first().index
        elif self.frequency == 'weekly':
            investment_dates = df.resample('W').first().index
        elif self.frequency == 'monthly':
            investment_dates = df.resample('M').first().index
        else:
            raise ValueError("Frequency must be 'daily', 'weekly', or 'monthly'")
        
        # Initialize investment tracking
        investments = []
        btc_holdings = 0
        total_invested = 0
        
        # Perform DCA strategy
        for date in investment_dates:
            if date in df.index:
                # Use the closing price for investment
                price = df.loc[date, 'close']
                btc_purchased = self.investment_amount / price
                btc_holdings += btc_purchased
                total_invested += self.investment_amount
                
                investments.append({
                    'date': date,
                    'price': price,
                    'investment': self.investment_amount,
                    'btc_purchased': btc_purchased,
                    'total_btc': btc_holdings,
                    'total_invested': total_invested,
                    'current_value': btc_holdings * price,
                    'roi': (btc_holdings * price / total_invested - 1) * 100 if total_invested > 0 else 0
                })
            
        # Convert to DataFrame
        results = pd.DataFrame(investments)
        
        return results
    
    def compare_with_lump_sum(self, results):
        """Compare DCA strategy with lump sum investment"""
        if results.empty:
            return None
        
        total_invested = results.iloc[-1]['total_invested']
        first_price = results.iloc[0]['price']
        last_price = results.iloc[-1]['price']
        
        lump_sum_btc = total_invested / first_price
        lump_sum_final_value = lump_sum_btc * last_price
        lump_sum_roi = (lump_sum_final_value / total_invested - 1) * 100
        
        dca_final_value = results.iloc[-1]['current_value']
        dca_roi = results.iloc[-1]['roi']
        
        comparison = {
            'Total Invested': total_invested,
            'DCA Final Value': dca_final_value,
            'DCA ROI': dca_roi,
            'Lump Sum Final Value': lump_sum_final_value,
            'Lump Sum ROI': lump_sum_roi,
            'Difference': dca_final_value - lump_sum_final_value,
            'Outperformer': 'DCA' if dca_final_value > lump_sum_final_value else 'Lump Sum'
        }
        
        return comparison
    
    def visualize_results(self, results):
        """Visualize the DCA performance"""
        if results.empty:
            print("No results to visualize")
            return
        
        # Set up the figure
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15), sharex=True)
        
        # Plot Bitcoin price
        ax1.plot(results['date'], results['price'], 'b-')
        ax1.set_title('Bitcoin Price')
        ax1.set_ylabel('USD')
        ax1.grid(True)
        
        # Plot accumulated BTC
        ax2.plot(results['date'], results['total_btc'], 'g-')
        ax2.set_title('Accumulated BTC')
        ax2.set_ylabel('BTC')
        ax2.grid(True)
        
        # Plot portfolio value and total invested
        ax3.plot(results['date'], results['current_value'], 'r-', label='Portfolio Value')
        ax3.plot(results['date'], results['total_invested'], 'k--', label='Total Invested')
        ax3.set_title('Portfolio Value vs. Total Invested')
        ax3.set_ylabel('USD')
        ax3.set_xlabel('Date')
        ax3.legend()
        ax3.grid(True)
        
        plt.tight_layout()
        plt.show()
        
        # ROI over time
        plt.figure(figsize=(12, 6))
        plt.plot(results['date'], results['roi'], 'g-')
        plt.title('Return on Investment Over Time')
        plt.ylabel('ROI (%)')
        plt.xlabel('Date')
        plt.grid(True)
        plt.axhline(y=0, color='r', linestyle='-')
        plt.tight_layout()
        plt.show()


def run_dca_simulation(data_path, investment_amount=100, frequency='monthly'):
    """Run a DCA simulation and display results"""
    bot = BitcoinDCABot(data_path, investment_amount, frequency)
    results = bot.simulate_dca()
    
    # Display summary statistics
    print(f"\n====== DCA SIMULATION SUMMARY ======")
    print(f"Strategy: ${investment_amount} {frequency} investment")
    print(f"Total periods: {len(results)}")
    
    if not results.empty:
        print(f"Total invested: ${results.iloc[-1]['total_invested']:.2f}")
        print(f"Final BTC holdings: {results.iloc[-1]['total_btc']:.8f}")
        print(f"Final portfolio value: ${results.iloc[-1]['current_value']:.2f}")
        print(f"Overall ROI: {results.iloc[-1]['roi']:.2f}%")
        
        # Compare with lump sum
        comparison = bot.compare_with_lump_sum(results)
        if comparison:
            print("\n====== DCA vs. LUMP SUM COMPARISON ======")
            for key, value in comparison.items():
                if isinstance(value, float):
                    print(f"{key}: {value:.2f}")
                else:
                    print(f"{key}: {value}")
        
        # Visualize results
        bot.visualize_results(results)
    else:
        print("No investment periods found in the data.")
    
    return results, bot


# EXECUTION CODE - Replace with your actual file path
if __name__ == "__main__":
    # Path to your Binance data CSV file
    data_path = "binance_data.csv"  # REPLACE THIS with your actual file path
    
    # Run the DCA simulation
    results, bot = run_dca_simulation(
        data_path=data_path,
        investment_amount=100,  # $100 per period
        frequency='monthly'     # Options: 'daily', 'weekly', 'monthly'
    )
    
    if not results.empty:
        # You can access the results dataframe for further analysis
        print("\nFirst few rows of the results:")
        print(results.head())
        
        # Example of additional analysis you might want to do
        print("\nBest month for investment (lowest price):")
        best_month = results.loc[results['price'].idxmin()]
        print(f"Date: {best_month['date']}, Price: ${best_month['price']:.2f}")
        
        print("\nWorst month for investment (highest price):")
        worst_month = results.loc[results['price'].idxmax()]
        print(f"Date: {worst_month['date']}, Price: ${worst_month['price']:.2f}")
if __name__ == "__main__":
    # Path to your Binance data CSV file
    data_path = "binance_data.csv"  # REPLACE THIS with your actual file path
    
    # Run the DCA simulation
    results, bot = run_dca_simulation(
        data_path=data_path,
        investment_amount=100,  # $100 per period
        frequency='monthly'     # Options: 'daily', 'weekly', 'monthly'
    )
    
    # You can access the results dataframe for further analysis
    print("\nFirst few rows of the results:")
    print(results.head())
    
    # Example of additional analysis you might want to do
    print("\nBest month for investment (lowest price):")
    best_month = results.loc[results['price'].idxmin()]
    print(f"Date: {best_month['date']}, Price: ${best_month['price']:.2f}")
    
    print("\nWorst month for investment (highest price):")
    worst_month = results.loc[results['price'].idxmax()]
    print(f"Date: {worst_month['date']}, Price: ${worst_month['price']:.2f}")


====== DCA SIMULATION SUMMARY ======
Strategy: $100 monthly investment
Total periods: 0
No investment periods found in the data.

====== DCA SIMULATION SUMMARY ======
Strategy: $100 monthly investment
Total periods: 0
No investment periods found in the data.

First few rows of the results:
Empty DataFrame
Columns: []
Index: []

Best month for investment (lowest price):


/var/folders/rl/039_ddsn1zjfd2t00_pz697w0000gn/T/ipykernel_53935/4047191904.py:230: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  investment_dates = df.resample('M').first().index
/var/folders/rl/039_ddsn1zjfd2t00_pz697w0000gn/T/ipykernel_53935/4047191904.py:230: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  investment_dates = df.resample('M').first().index


KeyError: 'Price'